In [1]:
 
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

Using TensorFlow backend.


In [2]:
from tensorflow.python.keras.backend import set_session
from tensorflow.python.keras.models import load_model

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

sess = tf.Session()
graph = tf.get_default_graph()

# IMPORTANT: models have to be loaded AFTER SETTING THE SESSION for keras! 
# Otherwise, their weights will be unavailable in the threads after the session there has been set
set_session(sess)

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
class LSGAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model(z, valid)
        # (!!!) Optimize w.r.t. MSE loss instead of crossentropy
        self.combined.compile(loss='mse', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        # (!!!) No softmax
        model.add(Dense(1))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise as generator input
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            #if epoch % sample_interval == 0:
                #self.sample_images(epoch)
            if epoch>=29000:
                self.sample_images(epoch)
            

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig(r"C:\Users\rishi\Documents\LSGAN Images\mnist_%d.png" % epoch)
        plt.close()


In [4]:
global sess
global graph
with graph.as_default():
    set_session(sess)
    gan = LSGAN()
    gan.train(epochs=30000, batch_size=32, sample_interval=200)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params:

c:\users\rishi\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.217458, acc.: 50.00%] [G loss: 1.546655]
1 [D loss: 0.815134, acc.: 51.56%] [G loss: 1.519244]
2 [D loss: 0.101150, acc.: 89.06%] [G loss: 1.438853]
3 [D loss: 0.072849, acc.: 95.31%] [G loss: 1.254242]
4 [D loss: 0.076860, acc.: 87.50%] [G loss: 1.026574]


c:\users\rishi\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


5 [D loss: 0.110396, acc.: 87.50%] [G loss: 1.116557]
6 [D loss: 0.074684, acc.: 90.62%] [G loss: 0.784145]
7 [D loss: 0.089250, acc.: 89.06%] [G loss: 1.105647]
8 [D loss: 0.121215, acc.: 79.69%] [G loss: 1.091111]
9 [D loss: 0.187178, acc.: 75.00%] [G loss: 1.116378]
10 [D loss: 0.439095, acc.: 53.12%] [G loss: 0.954286]
11 [D loss: 0.882448, acc.: 39.06%] [G loss: 1.213121]
12 [D loss: 1.107864, acc.: 40.62%] [G loss: 1.094201]
13 [D loss: 0.442695, acc.: 48.44%] [G loss: 1.139102]
14 [D loss: 0.174229, acc.: 76.56%] [G loss: 1.189992]
15 [D loss: 0.084051, acc.: 89.06%] [G loss: 1.120837]
16 [D loss: 0.090406, acc.: 90.62%] [G loss: 1.132020]
17 [D loss: 0.092694, acc.: 87.50%] [G loss: 0.919563]
18 [D loss: 0.111787, acc.: 89.06%] [G loss: 1.082492]
19 [D loss: 0.105074, acc.: 82.81%] [G loss: 1.216033]
20 [D loss: 0.083110, acc.: 90.62%] [G loss: 1.061370]
21 [D loss: 0.118039, acc.: 89.06%] [G loss: 1.177582]
22 [D loss: 0.090553, acc.: 90.62%] [G loss: 1.024061]
23 [D loss: 0.1

...

29418 [D loss: 0.229053, acc.: 62.50%] [G loss: 0.514875]
29419 [D loss: 0.213070, acc.: 68.75%] [G loss: 0.463601]
29420 [D loss: 0.243284, acc.: 57.81%] [G loss: 0.489629]
29421 [D loss: 0.169351, acc.: 79.69%] [G loss: 0.532956]
29422 [D loss: 0.243055, acc.: 54.69%] [G loss: 0.490466]
29423 [D loss: 0.227331, acc.: 62.50%] [G loss: 0.452313]
29424 [D loss: 0.219621, acc.: 67.19%] [G loss: 0.431213]
29425 [D loss: 0.234060, acc.: 65.62%] [G loss: 0.453068]
29426 [D loss: 0.195080, acc.: 67.19%] [G loss: 0.408053]
29427 [D loss: 0.215683, acc.: 65.62%] [G loss: 0.438123]
29428 [D loss: 0.242153, acc.: 50.00%] [G loss: 0.504838]
29429 [D loss: 0.197031, acc.: 65.62%] [G loss: 0.472955]
29430 [D loss: 0.230591, acc.: 56.25%] [G loss: 0.502577]
29431 [D loss: 0.217365, acc.: 64.06%] [G loss: 0.435171]
29432 [D loss: 0.220245, acc.: 62.50%] [G loss: 0.433669]
29433 [D loss: 0.247218, acc.: 60.94%] [G loss: 0.378879]
29434 [D loss: 0.222649, acc.: 68.75%] [G loss: 0.477360]
29435 [D loss:

29560 [D loss: 0.224868, acc.: 64.06%] [G loss: 0.419523]
29561 [D loss: 0.219201, acc.: 60.94%] [G loss: 0.492791]
29562 [D loss: 0.163369, acc.: 79.69%] [G loss: 0.489119]
29563 [D loss: 0.220004, acc.: 62.50%] [G loss: 0.415235]
29564 [D loss: 0.210808, acc.: 62.50%] [G loss: 0.419539]
29565 [D loss: 0.224088, acc.: 68.75%] [G loss: 0.535702]
29566 [D loss: 0.206070, acc.: 65.62%] [G loss: 0.446081]
29567 [D loss: 0.215109, acc.: 60.94%] [G loss: 0.451886]
29568 [D loss: 0.225313, acc.: 64.06%] [G loss: 0.482006]
29569 [D loss: 0.204225, acc.: 70.31%] [G loss: 0.450933]
29570 [D loss: 0.219282, acc.: 59.38%] [G loss: 0.465244]
29571 [D loss: 0.213083, acc.: 68.75%] [G loss: 0.437272]
29572 [D loss: 0.201196, acc.: 70.31%] [G loss: 0.452186]
29573 [D loss: 0.205802, acc.: 64.06%] [G loss: 0.487202]
29574 [D loss: 0.218547, acc.: 60.94%] [G loss: 0.481740]
29575 [D loss: 0.224500, acc.: 68.75%] [G loss: 0.529690]
29576 [D loss: 0.220183, acc.: 56.25%] [G loss: 0.444824]
29577 [D loss:

29702 [D loss: 0.236466, acc.: 57.81%] [G loss: 0.431277]
29703 [D loss: 0.236947, acc.: 53.12%] [G loss: 0.398643]
29704 [D loss: 0.177913, acc.: 75.00%] [G loss: 0.433523]
29705 [D loss: 0.216150, acc.: 67.19%] [G loss: 0.489191]
29706 [D loss: 0.216290, acc.: 64.06%] [G loss: 0.481476]
29707 [D loss: 0.213042, acc.: 70.31%] [G loss: 0.500890]
29708 [D loss: 0.238155, acc.: 67.19%] [G loss: 0.462636]
29709 [D loss: 0.219601, acc.: 65.62%] [G loss: 0.474220]
29710 [D loss: 0.227215, acc.: 60.94%] [G loss: 0.477240]
29711 [D loss: 0.219323, acc.: 67.19%] [G loss: 0.484531]
29712 [D loss: 0.202869, acc.: 70.31%] [G loss: 0.471371]
29713 [D loss: 0.209627, acc.: 59.38%] [G loss: 0.439519]
29714 [D loss: 0.207071, acc.: 62.50%] [G loss: 0.486530]
29715 [D loss: 0.205505, acc.: 71.88%] [G loss: 0.476389]
29716 [D loss: 0.212980, acc.: 65.62%] [G loss: 0.458949]
29717 [D loss: 0.229457, acc.: 56.25%] [G loss: 0.439783]
29718 [D loss: 0.288594, acc.: 45.31%] [G loss: 0.458328]
29719 [D loss:

29844 [D loss: 0.254373, acc.: 56.25%] [G loss: 0.358085]
29845 [D loss: 0.206813, acc.: 64.06%] [G loss: 0.461028]
29846 [D loss: 0.204804, acc.: 65.62%] [G loss: 0.508430]
29847 [D loss: 0.261282, acc.: 60.94%] [G loss: 0.449536]
29848 [D loss: 0.165747, acc.: 75.00%] [G loss: 0.456111]
29849 [D loss: 0.238136, acc.: 57.81%] [G loss: 0.482282]
29850 [D loss: 0.156943, acc.: 81.25%] [G loss: 0.445197]
29851 [D loss: 0.205557, acc.: 67.19%] [G loss: 0.468258]
29852 [D loss: 0.223049, acc.: 60.94%] [G loss: 0.435845]
29853 [D loss: 0.231693, acc.: 56.25%] [G loss: 0.473885]
29854 [D loss: 0.215480, acc.: 75.00%] [G loss: 0.443418]
29855 [D loss: 0.240105, acc.: 57.81%] [G loss: 0.448068]
29856 [D loss: 0.217076, acc.: 60.94%] [G loss: 0.467356]
29857 [D loss: 0.223686, acc.: 65.62%] [G loss: 0.431296]
29858 [D loss: 0.202047, acc.: 68.75%] [G loss: 0.464252]
29859 [D loss: 0.207540, acc.: 71.88%] [G loss: 0.492022]
29860 [D loss: 0.207338, acc.: 73.44%] [G loss: 0.490911]
29861 [D loss:

29986 [D loss: 0.191691, acc.: 65.62%] [G loss: 0.481260]
29987 [D loss: 0.220443, acc.: 68.75%] [G loss: 0.455525]
29988 [D loss: 0.206425, acc.: 62.50%] [G loss: 0.491035]
29989 [D loss: 0.220141, acc.: 65.62%] [G loss: 0.473100]
29990 [D loss: 0.205516, acc.: 70.31%] [G loss: 0.489861]
29991 [D loss: 0.181437, acc.: 78.12%] [G loss: 0.471991]
29992 [D loss: 0.239045, acc.: 56.25%] [G loss: 0.424719]
29993 [D loss: 0.227317, acc.: 60.94%] [G loss: 0.433284]
29994 [D loss: 0.221955, acc.: 64.06%] [G loss: 0.421033]
29995 [D loss: 0.225291, acc.: 59.38%] [G loss: 0.478631]
29996 [D loss: 0.214615, acc.: 67.19%] [G loss: 0.426528]
29997 [D loss: 0.204395, acc.: 68.75%] [G loss: 0.460936]
29998 [D loss: 0.214628, acc.: 54.69%] [G loss: 0.446764]
29999 [D loss: 0.229028, acc.: 57.81%] [G loss: 0.466374]


In [5]:
import numpy as np

In [5]:
def generate_images(cnt):
    c=1
    while(c <= cnt):
        noise = np.random.normal(0, 1, (1, 100))
        gen_imgs = gan.generator.predict(noise)
        gen_imgs = 0.5 * gen_imgs + 0.5
        plt.imshow(gen_imgs[0, :,:,0], cmap='gray')
        plt.axis('off')
        plt.savefig(r"C:\Users\rishi\Documents\LSGAN Images\generated\lsgan_{}.jpg".format(c), bbox_inches='tight', pad_inches=-0.0005)
        print("Image {} saved.".format(c))
        c+=1
        plt.close()
generate_images(10000)

Image 1 saved.
Image 2 saved.
Image 3 saved.
Image 4 saved.
Image 5 saved.
Image 6 saved.
Image 7 saved.
Image 8 saved.
Image 9 saved.
Image 10 saved.
Image 11 saved.
Image 12 saved.
Image 13 saved.
Image 14 saved.
Image 15 saved.
Image 16 saved.
Image 17 saved.
Image 18 saved.
Image 19 saved.
Image 20 saved.
Image 21 saved.
Image 22 saved.
Image 23 saved.
Image 24 saved.
Image 25 saved.
Image 26 saved.
Image 27 saved.
Image 28 saved.
Image 29 saved.
Image 30 saved.
Image 31 saved.
Image 32 saved.
Image 33 saved.
Image 34 saved.
Image 35 saved.
Image 36 saved.
Image 37 saved.
Image 38 saved.
Image 39 saved.
Image 40 saved.
Image 41 saved.
Image 42 saved.
Image 43 saved.
Image 44 saved.
Image 45 saved.
Image 46 saved.
Image 47 saved.
Image 48 saved.
Image 49 saved.
Image 50 saved.
Image 51 saved.
Image 52 saved.
Image 53 saved.
Image 54 saved.
Image 55 saved.
Image 56 saved.
Image 57 saved.
Image 58 saved.
Image 59 saved.
Image 60 saved.
Image 61 saved.
Image 62 saved.
Image 63 saved.
I

Image 490 saved.
Image 491 saved.
Image 492 saved.
Image 493 saved.
Image 494 saved.
Image 495 saved.
Image 496 saved.
Image 497 saved.
Image 498 saved.
Image 499 saved.
Image 500 saved.
Image 501 saved.
Image 502 saved.
Image 503 saved.
Image 504 saved.
Image 505 saved.
Image 506 saved.
Image 507 saved.
Image 508 saved.
Image 509 saved.
Image 510 saved.
Image 511 saved.
Image 512 saved.
Image 513 saved.
Image 514 saved.
Image 515 saved.
Image 516 saved.
Image 517 saved.
Image 518 saved.
Image 519 saved.
Image 520 saved.
Image 521 saved.
Image 522 saved.
Image 523 saved.
Image 524 saved.
Image 525 saved.
Image 526 saved.
Image 527 saved.
Image 528 saved.
Image 529 saved.
Image 530 saved.
Image 531 saved.
Image 532 saved.
Image 533 saved.
Image 534 saved.
Image 535 saved.
Image 536 saved.
Image 537 saved.
Image 538 saved.
Image 539 saved.
Image 540 saved.
Image 541 saved.
Image 542 saved.
Image 543 saved.
Image 544 saved.
Image 545 saved.
Image 546 saved.
Image 547 saved.
Image 548 save

Image 973 saved.
Image 974 saved.
Image 975 saved.
Image 976 saved.
Image 977 saved.
Image 978 saved.
Image 979 saved.
Image 980 saved.
Image 981 saved.
Image 982 saved.
Image 983 saved.
Image 984 saved.
Image 985 saved.
Image 986 saved.
Image 987 saved.
Image 988 saved.
Image 989 saved.
Image 990 saved.
Image 991 saved.
Image 992 saved.
Image 993 saved.
Image 994 saved.
Image 995 saved.
Image 996 saved.
Image 997 saved.
Image 998 saved.
Image 999 saved.
Image 1000 saved.
Image 1001 saved.
Image 1002 saved.
Image 1003 saved.
Image 1004 saved.
Image 1005 saved.
Image 1006 saved.
Image 1007 saved.
Image 1008 saved.
Image 1009 saved.
Image 1010 saved.
Image 1011 saved.
Image 1012 saved.
Image 1013 saved.
Image 1014 saved.
Image 1015 saved.
Image 1016 saved.
Image 1017 saved.
Image 1018 saved.
Image 1019 saved.
Image 1020 saved.
Image 1021 saved.
Image 1022 saved.
Image 1023 saved.
Image 1024 saved.
Image 1025 saved.
Image 1026 saved.
Image 1027 saved.
Image 1028 saved.
Image 1029 saved.
I

Image 1436 saved.
Image 1437 saved.
Image 1438 saved.
Image 1439 saved.
Image 1440 saved.
Image 1441 saved.
Image 1442 saved.
Image 1443 saved.
Image 1444 saved.
Image 1445 saved.
Image 1446 saved.
Image 1447 saved.
Image 1448 saved.
Image 1449 saved.
Image 1450 saved.
Image 1451 saved.
Image 1452 saved.
Image 1453 saved.
Image 1454 saved.
Image 1455 saved.
Image 1456 saved.
Image 1457 saved.
Image 1458 saved.
Image 1459 saved.
Image 1460 saved.
Image 1461 saved.
Image 1462 saved.
Image 1463 saved.
Image 1464 saved.
Image 1465 saved.
Image 1466 saved.
Image 1467 saved.
Image 1468 saved.
Image 1469 saved.
Image 1470 saved.
Image 1471 saved.
Image 1472 saved.
Image 1473 saved.
Image 1474 saved.
Image 1475 saved.
Image 1476 saved.
Image 1477 saved.
Image 1478 saved.
Image 1479 saved.
Image 1480 saved.
Image 1481 saved.
Image 1482 saved.
Image 1483 saved.
Image 1484 saved.
Image 1485 saved.
Image 1486 saved.
Image 1487 saved.
Image 1488 saved.
Image 1489 saved.
Image 1490 saved.
Image 1491

Image 1896 saved.
Image 1897 saved.
Image 1898 saved.
Image 1899 saved.
Image 1900 saved.
Image 1901 saved.
Image 1902 saved.
Image 1903 saved.
Image 1904 saved.
Image 1905 saved.
Image 1906 saved.
Image 1907 saved.
Image 1908 saved.
Image 1909 saved.
Image 1910 saved.
Image 1911 saved.
Image 1912 saved.
Image 1913 saved.
Image 1914 saved.
Image 1915 saved.
Image 1916 saved.
Image 1917 saved.
Image 1918 saved.
Image 1919 saved.
Image 1920 saved.
Image 1921 saved.
Image 1922 saved.
Image 1923 saved.
Image 1924 saved.
Image 1925 saved.
Image 1926 saved.
Image 1927 saved.
Image 1928 saved.
Image 1929 saved.
Image 1930 saved.
Image 1931 saved.
Image 1932 saved.
Image 1933 saved.
Image 1934 saved.
Image 1935 saved.
Image 1936 saved.
Image 1937 saved.
Image 1938 saved.
Image 1939 saved.
Image 1940 saved.
Image 1941 saved.
Image 1942 saved.
Image 1943 saved.
Image 1944 saved.
Image 1945 saved.
Image 1946 saved.
Image 1947 saved.
Image 1948 saved.
Image 1949 saved.
Image 1950 saved.
Image 1951

...

Image 7846 saved.
Image 7847 saved.
Image 7848 saved.
Image 7849 saved.
Image 7850 saved.
Image 7851 saved.
Image 7852 saved.
Image 7853 saved.
Image 7854 saved.
Image 7855 saved.
Image 7856 saved.
Image 7857 saved.
Image 7858 saved.
Image 7859 saved.
Image 7860 saved.
Image 7861 saved.
Image 7862 saved.
Image 7863 saved.
Image 7864 saved.
Image 7865 saved.
Image 7866 saved.
Image 7867 saved.
Image 7868 saved.
Image 7869 saved.
Image 7870 saved.
Image 7871 saved.
Image 7872 saved.
Image 7873 saved.
Image 7874 saved.
Image 7875 saved.
Image 7876 saved.
Image 7877 saved.
Image 7878 saved.
Image 7879 saved.
Image 7880 saved.
Image 7881 saved.
Image 7882 saved.
Image 7883 saved.
Image 7884 saved.
Image 7885 saved.
Image 7886 saved.
Image 7887 saved.
Image 7888 saved.
Image 7889 saved.
Image 7890 saved.
Image 7891 saved.
Image 7892 saved.
Image 7893 saved.
Image 7894 saved.
Image 7895 saved.
Image 7896 saved.
Image 7897 saved.
Image 7898 saved.
Image 7899 saved.
Image 7900 saved.
Image 7901

Image 8304 saved.
Image 8305 saved.
Image 8306 saved.
Image 8307 saved.
Image 8308 saved.
Image 8309 saved.
Image 8310 saved.
Image 8311 saved.
Image 8312 saved.
Image 8313 saved.
Image 8314 saved.
Image 8315 saved.
Image 8316 saved.
Image 8317 saved.
Image 8318 saved.
Image 8319 saved.
Image 8320 saved.
Image 8321 saved.
Image 8322 saved.
Image 8323 saved.
Image 8324 saved.
Image 8325 saved.
Image 8326 saved.
Image 8327 saved.
Image 8328 saved.
Image 8329 saved.
Image 8330 saved.
Image 8331 saved.
Image 8332 saved.
Image 8333 saved.
Image 8334 saved.
Image 8335 saved.
Image 8336 saved.
Image 8337 saved.
Image 8338 saved.
Image 8339 saved.
Image 8340 saved.
Image 8341 saved.
Image 8342 saved.
Image 8343 saved.
Image 8344 saved.
Image 8345 saved.
Image 8346 saved.
Image 8347 saved.
Image 8348 saved.
Image 8349 saved.
Image 8350 saved.
Image 8351 saved.
Image 8352 saved.
Image 8353 saved.
Image 8354 saved.
Image 8355 saved.
Image 8356 saved.
Image 8357 saved.
Image 8358 saved.
Image 8359

Image 8760 saved.
Image 8761 saved.
Image 8762 saved.
Image 8763 saved.
Image 8764 saved.
Image 8765 saved.
Image 8766 saved.
Image 8767 saved.
Image 8768 saved.
Image 8769 saved.
Image 8770 saved.
Image 8771 saved.
Image 8772 saved.
Image 8773 saved.
Image 8774 saved.
Image 8775 saved.
Image 8776 saved.
Image 8777 saved.
Image 8778 saved.
Image 8779 saved.
Image 8780 saved.
Image 8781 saved.
Image 8782 saved.
Image 8783 saved.
Image 8784 saved.
Image 8785 saved.
Image 8786 saved.
Image 8787 saved.
Image 8788 saved.
Image 8789 saved.
Image 8790 saved.
Image 8791 saved.
Image 8792 saved.
Image 8793 saved.
Image 8794 saved.
Image 8795 saved.
Image 8796 saved.
Image 8797 saved.
Image 8798 saved.
Image 8799 saved.
Image 8800 saved.
Image 8801 saved.
Image 8802 saved.
Image 8803 saved.
Image 8804 saved.
Image 8805 saved.
Image 8806 saved.
Image 8807 saved.
Image 8808 saved.
Image 8809 saved.
Image 8810 saved.
Image 8811 saved.
Image 8812 saved.
Image 8813 saved.
Image 8814 saved.
Image 8815

Image 9215 saved.
Image 9216 saved.
Image 9217 saved.
Image 9218 saved.
Image 9219 saved.
Image 9220 saved.
Image 9221 saved.
Image 9222 saved.
Image 9223 saved.
Image 9224 saved.
Image 9225 saved.
Image 9226 saved.
Image 9227 saved.
Image 9228 saved.
Image 9229 saved.
Image 9230 saved.
Image 9231 saved.
Image 9232 saved.
Image 9233 saved.
Image 9234 saved.
Image 9235 saved.
Image 9236 saved.
Image 9237 saved.
Image 9238 saved.
Image 9239 saved.
Image 9240 saved.
Image 9241 saved.
Image 9242 saved.
Image 9243 saved.
Image 9244 saved.
Image 9245 saved.
Image 9246 saved.
Image 9247 saved.
Image 9248 saved.
Image 9249 saved.
Image 9250 saved.
Image 9251 saved.
Image 9252 saved.
Image 9253 saved.
Image 9254 saved.
Image 9255 saved.
Image 9256 saved.
Image 9257 saved.
Image 9258 saved.
Image 9259 saved.
Image 9260 saved.
Image 9261 saved.
Image 9262 saved.
Image 9263 saved.
Image 9264 saved.
Image 9265 saved.
Image 9266 saved.
Image 9267 saved.
Image 9268 saved.
Image 9269 saved.
Image 9270

Image 9671 saved.
Image 9672 saved.
Image 9673 saved.
Image 9674 saved.
Image 9675 saved.
Image 9676 saved.
Image 9677 saved.
Image 9678 saved.
Image 9679 saved.
Image 9680 saved.
Image 9681 saved.
Image 9682 saved.
Image 9683 saved.
Image 9684 saved.
Image 9685 saved.
Image 9686 saved.
Image 9687 saved.
Image 9688 saved.
Image 9689 saved.
Image 9690 saved.
Image 9691 saved.
Image 9692 saved.
Image 9693 saved.
Image 9694 saved.
Image 9695 saved.
Image 9696 saved.
Image 9697 saved.
Image 9698 saved.
Image 9699 saved.
Image 9700 saved.
Image 9701 saved.
Image 9702 saved.
Image 9703 saved.
Image 9704 saved.
Image 9705 saved.
Image 9706 saved.
Image 9707 saved.
Image 9708 saved.
Image 9709 saved.
Image 9710 saved.
Image 9711 saved.
Image 9712 saved.
Image 9713 saved.
Image 9714 saved.
Image 9715 saved.
Image 9716 saved.
Image 9717 saved.
Image 9718 saved.
Image 9719 saved.
Image 9720 saved.
Image 9721 saved.
Image 9722 saved.
Image 9723 saved.
Image 9724 saved.
Image 9725 saved.
Image 9726

In [7]:
# serialize model to JSON
from keras.models import model_from_json
model_json = gan.to_json()
with open(r"C:\Users\rishi\Documents\LSGAN Images\lsgan.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(r"C:\Users\rishi\Documents\LSGAN Images\lsgan.h5")
print("Saved model to disk")
 


AttributeError: 'LSGAN' object has no attribute 'to_json'

In [ ]:
# later...
 
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")